# Data Preprocessing

In this file, we will :

- Load the three datasets (`train`, `test`, `evaluate`)
- Process them :
  - Resize and format the images
  - Apply the masks on the images
- Save the resulting numpy arrays in npz files


In [1]:
from datasets import load_dataset

train_dataset = load_dataset("Az-r-ow/chest_xray", split="train")
test_dataset = load_dataset("Az-r-ow/chest_xray", split="test")
eval_dataset = load_dataset("Az-r-ow/chest_xray", split="validation")

In [2]:
from utils.helpers import format_dataset

train_labels = train_dataset.features["label"].names
test_labels = test_dataset.features["label"].names
eval_labels = eval_dataset.features["label"].names